# Preprocessing oldlisting_buy_3

In [1]:
import sys, os
sys.path.append(os.path.abspath('../'))
from scripts.utils import create_dir, get_runtime
import time 
start_time = time.time()

import pandas as pd
import re
import ast

In [2]:
# Load your dataset (replace with the correct path to your CSV file)
file_path = "../data/landing/oldlistings_buy/oldlistings_buy_3.csv"
df = pd.read_csv(file_path)

# Helper functions to extract and process data
def expand_rented_prices(row):
    try:
        rent_list = ast.literal_eval(row['rented_prices'])
        rows = []
        for rent in rent_list:
            new_row = row.copy()
            new_row['rented_price'] = rent.get('price', None)
            new_row['date'] = rent.get('date', None)
            rows.append(new_row)
        return pd.DataFrame(rows)
    except (ValueError, SyntaxError):
        return pd.DataFrame([row])

def extract_from_meta_data(meta_data_str, label):
    try:
        meta_list = ast.literal_eval(meta_data_str)
        for item in meta_list:
            if item.get('label') == label:
                return item.get('quantity', None)
    except Exception:
        return None

# Apply the process to expand the first 100 rows (or all rows if needed)
expanded_rows = pd.concat([expand_rented_prices(row) for _, row in df.iterrows()], ignore_index=True)

# Extract meta_data columns for bed, bath, car, land, type
expanded_rows['bed'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'bed'))
expanded_rows['bath'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'bath'))
expanded_rows['car'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'car'))
expanded_rows['land'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'land'))
expanded_rows['type'] = expanded_rows['meta_data'].apply(lambda x: extract_from_meta_data(x, 'type'))

# Keep only relevant columns
final_expanded_df = expanded_rows[['lat', 'lng', 'address', 'bed', 'bath', 'car', 'land', 'type', 'rented_price', 'date']]

# Optionally, print or view the dataframe
# print(final_expanded_df)

In [3]:
digit_translation_table = str.maketrans({
    '０': '0', '１': '1', '２': '2', '３': '3', '４': '4', '５': '5', '６': '6', '７': '7', '８': '8', '９': '9',
    '𝟶': '0', '𝟷': '1', '𝟸': '2', '𝟹': '3', '𝟺': '4', '𝟻': '5', '𝟼': '6', '𝟽': '7', '𝟾': '8', '𝟿': '9',
    '𝟢': '0', '𝟣': '1', '𝟤': '2', '𝟥': '3', '𝟦': '4', '𝟧': '5', '𝟨': '6', '𝟩': '7', '𝟪': '8', '𝟫': '9',
    '𝟘': '0', '𝟙': '1', '𝟚': '2', '𝟛': '3', '𝟜': '4', '𝟝': '5', '𝟞': '6', '𝟟': '7', '𝟠': '8', '𝟡': '9',
    '𝞾': '0', '𝞿': '1', '𝟁': '3'
})

In [4]:
# Function to clean rented_prices
final_expanded_df['property_price_cleaned'] = (
    final_expanded_df['rented_price']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    .str.replace(r'<SPAN>', '', regex=False)           # Remove <SPAN> tag
    .str.replace(r'</SPAN>', '', regex=False)          # Remove </SPAN> tag
    .str.translate(digit_translation_table)           # Translate full-width digits to half-width digits
)

/tmp/ipykernel_89422/1370685432.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['property_price_cleaned'] = (


In [5]:
# Replace "O" with "0" in rented_price_cleaned
final_expanded_df['property_price_cleaned'] = final_expanded_df['property_price_cleaned'].str.replace('O', '0')

/tmp/ipykernel_89422/2979407506.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['property_price_cleaned'] = final_expanded_df['property_price_cleaned'].str.replace('O', '0')


In [6]:
# Replace all non-numeric characters with NaN from rented_price_cleaned except for commas and "$" signs and "-" signs
final_expanded_df['property_price_cleaned'] = final_expanded_df['property_price_cleaned'].str.replace(r'[^0-9,$-]', '', regex=True)

/tmp/ipykernel_89422/2754561085.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['property_price_cleaned'] = final_expanded_df['property_price_cleaned'].str.replace(r'[^0-9,$-]', '', regex=True)


In [7]:
# Function to clean and handle range prices by calculating the average
def clean_price(price):
    if pd.isna(price):
        return price  # Return NaN as is
    # Handle price ranges like "$425,000-$455,000"
    range_match = re.match(r"\$(\d+,\d+)-\$(\d+,\d+)", price)
    if range_match:
        low_price = int(range_match.group(1).replace(',', ''))
        high_price = int(range_match.group(2).replace(',', ''))
        return (low_price + high_price) / 2  # Return the average of the range
    # Handle normal prices
    price_cleaned = re.sub(r'[^\d]', '', price)
    return int(price_cleaned) if price_cleaned.isdigit() else None

In [8]:
final_expanded_df['property_price_cleaned'] = [clean_price(price) for price in final_expanded_df['property_price_cleaned']]

/tmp/ipykernel_89422/4207001755.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['property_price_cleaned'] = [clean_price(price) for price in final_expanded_df['property_price_cleaned']]


In [9]:
final_expanded_df['date'] = pd.to_datetime(final_expanded_df['date'], errors='coerce')

/tmp/ipykernel_89422/483767387.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  final_expanded_df['date'] = pd.to_datetime(final_expanded_df['date'], errors='coerce')
/tmp/ipykernel_89422/483767387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['date'] = pd.to_datetime(final_expanded_df['date'], errors='coerce')


In [10]:
# Function to clean "bed"
final_expanded_df['bed_cleaned'] = (
    final_expanded_df['bed']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    .str.replace(r'<SPAN>', '', regex=False)           # Remove <SPAN> tag
    .str.replace(r'</SPAN>', '', regex=False)          # Remove </SPAN> tag
    .str.translate(digit_translation_table)            # Translate full-width digits to half-width digits
)

/tmp/ipykernel_89422/2217788108.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_expanded_df['bed_cleaned'] = (


In [11]:
# Function to clean "bath"
final_expanded_df['bath_cleaned'] = (
    final_expanded_df['bath']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    .str.replace(r'<SPAN>', '', regex=False)           # Remove <SPAN> tag
    .str.replace(r'</SPAN>', '', regex=False)          # Remove </SPAN> tag
    .str.translate(digit_translation_table)
)

In [12]:
# Function to clean "car"
final_expanded_df['car_cleaned'] = (
    final_expanded_df['car']
    .str.replace(r'\u200b', '', regex=False)           # Remove zero-width space
    .str.replace(r'\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace(r'<span>', '', regex=False)           # Remove <span> tag
    .str.replace(r'</span>', '', regex=False)          # Remove </span> tag
    .str.replace(r'<SPAN>', '', regex=False)           # Remove <SPAN> tag
    .str.replace(r'</SPAN>', '', regex=False)          # Remove </SPAN> tag
    .str.translate(digit_translation_table)
)

In [13]:
letter_digit_translation_table = str.maketrans({
    # Uppercase letters
    '𝙰': 'A', '𝙱': 'B', '𝙲': 'C', '𝙳': 'D', '𝙴': 'E', '𝙵': 'F', '𝙶': 'G', '𝙷': 'H', '𝙸': 'I', 
    '𝙹': 'J', '𝙺': 'K', '𝙻': 'L', '𝚀': 'Q', '𝚁': 'R', '𝚂': 'S', '𝚃': 'T', '𝚄': 'U', '𝚅': 'V', 
    '𝚆': 'W', '𝚇': 'X', '𝚈': 'Y', '𝚉': 'Z',
    # Lowercase letters
    '𝚊': 'a', '𝚋': 'b', '𝚌': 'c', '𝚍': 'd', '𝚎': 'e', 
    '𝚏': 'f', '𝚐': 'g', '𝚑': 'h', '𝚒': 'i', '𝚓': 'j', '𝚔': 'k', '𝚕': 'l', '𝚖': 'm', '𝚗': 'n', 
    '𝚘': 'o', '𝚙': 'p', '𝚚': 'q', '𝚛': 'r', '𝚜': 's', '𝚝': 't', '𝚞': 'u', '𝚟': 'v', '𝚠': 'w', 
    '𝚡': 'x', '𝚢': 'y', '𝚣': 'z',
    # Full-width and other non-standard digits
    '０': '0', '１': '1', '２': '2', '３': '3', '４': '4', '５': '5', '６': '6', '７': '7', '８': '8', '９': '9',
    '𝟶': '0', '𝟷': '1', '𝟸': '2', '𝟹': '3', '𝟺': '4', '𝟻': '5', '𝟼': '6', '𝟽': '7', '𝟾': '8', '𝟿': '9',
    '𝟢': '0', '𝟣': '1', '𝟤': '2', '𝟥': '3', '𝟦': '4', '𝟧': '5', '𝟨': '6', '𝟩': '7', '𝟪': '8', '𝟫': '9',
    '𝟘': '0', '𝟙': '1', '𝟚': '2', '𝟛': '3', '𝟜': '4', '𝟝': '5', '𝟞': '6', '𝟟': '7', '𝟠': '8', '𝟡': '9',
    '𝞾': '0', '𝞿': '1', '𝟁': '3'
})

In [14]:
# Function to clean "address"
final_expanded_df['address_cleaned'] = (
    final_expanded_df['address']
    .str.replace('\u200b', '', regex=False)           # Remove zero-width space
    .str.replace('\xa0', '', regex=False)             # Remove non-breaking space
    .str.replace('<span>', '', regex=False)           # Remove <span> tag
    .str.replace('</span>', '', regex=False)          # Remove </span> tag
    .str.replace('<SPAN>', '', regex=False)           # Remove <SPAN> tag
    .str.replace('</SPAN>', '', regex=False)          # Remove </SPAN> tag
    .str.translate(letter_digit_translation_table)     # Convert full-width letters to ASCII letters
)

In [15]:
# drop some columns
final_expanded_df = final_expanded_df.drop(columns=['rented_price', 'bed', 'bath', 'car', 'land', 'address'])
 
# Extract year from date
final_expanded_df['year'] = final_expanded_df['date'].dt.year

# Extract suburb from address_cleaned, which is all text after the last comma, removing leading and trailing whitespaces
final_expanded_df['suburb'] = final_expanded_df['address_cleaned'].str.split(',').str[-1].str.strip()

# Remove all type that are not 'House' or 'Unit/ampt'
final_expanded_df = final_expanded_df[final_expanded_df['type'].isin(['House', 'Unit/apmt'])]

# Remove all rows with NaN in 'property_price_cleaned'
final_expanded_df = final_expanded_df[final_expanded_df['property_price_cleaned'].notna()]

In [16]:
# Extract suburb from address_cleaned, which is all text after the last comma
final_expanded_df['suburb'] = final_expanded_df['address_cleaned'].str.rsplit(',').str[-1]

In [17]:
final_expanded_df['bed_cleaned'] = pd.to_numeric(final_expanded_df['bed_cleaned'], errors='coerce')
final_expanded_df['bath_cleaned'] = pd.to_numeric(final_expanded_df['bath_cleaned'], errors='coerce')
final_expanded_df['car_cleaned'] = pd.to_numeric(final_expanded_df['car_cleaned'], errors='coerce')

In [18]:
# In the column 'property_price_cleaned', remove the rows that digits are more than 10
final_expanded_df = final_expanded_df[final_expanded_df['property_price_cleaned'].astype(str).str.len() <= 10]
final_expanded_df

,lat,lng,type,date,property_price_cleaned,bed_cleaned,bath_cleaned,car_cleaned,address_cleaned,year,suburb
0,NaN,NaN,House,2022-08-01,0.0,5.0,2.0,4.0,"36 LΑWLΕSS DR, CRANBOURNE NORTH",2022.0,CRANBOURNE NORTH
2,NaN,NaN,House,2022-08-01,4500000.0,5.0,2.0,4.0,"36 LΑWLΕSS DRIVΕ, CRANBOURNE NORTH",2022.0,CRANBOURNE NORTH
3,NaN,NaN,House,2022-08-01,787500.0,5.0,2.0,4.0,"36 LΑWLΕSS DRIVΕ, CRANBOURNE NORTH",2022.0,CRANBOURNE NORTH
5,NaN,NaN,House,2022-08-01,720000.0,3.0,2.0,2.0,"14 FIELDSTONE CRESCENT, СRΑNВОՍRNΕ NОRTH",2022.0,СRΑNВОՍRNΕ NОRTH
6,NaN,NaN,House,2022-08-01,720000.0,3.0,2.0,2.0,"14 FIELDSTONE CRESCENT, СRΑNВОՍRNΕ NОRTH",2022.0,СRΑNВОՍRNΕ NОRTH
...,...,...,...,...,...,...,...,...,...,...,...
519855,NaN,NaN,House,2021-07-01,725000.0,3.0,1.0,1.0,"5 NEPEAN STREET, WΑTSОNIΑ",2021.0,WΑTSОNIΑ
519856,NaN,NaN,House,2021-06-01,700000.0,3.0,1.0,1.0,"5 NEPEAN STREET, WΑTSОNIΑ",2021.0,WΑTSОNIΑ
519859,NaN,NaN,House,2021-06-01,692500.0,3.0,1.0,2.0,"20 PETERS STREET, WATSONIA",2021.0,WATSONIA
519861,NaN,NaN,House,2021-06-01,725000.0,3.0,1.0,1.0,"40 YERRAWA DRIVE, WATSONIA",2021.0,WATSONIA


In [19]:
create_dir("../data/raw/oldlistings_buy/")
final_expanded_df.to_csv("../data/raw/oldlistings_buy/oldlistings_buy_3.csv", index=False)

Directory already exists: ../data/raw/oldlistings_buy/



In [20]:
# Calculate average property price by year, suburb
final_expanded_df_avg = final_expanded_df.groupby(['year', 'suburb']).agg(
    avg_property_price=('property_price_cleaned', 'mean'),
).reset_index()
final_expanded_df_avg

,year,suburb,avg_property_price
0,2006.0,ALTONA MEADOWS,528954.545455
1,2006.0,ALTONA MΕΑDОWS,199000.000000
2,2006.0,ALTONA NORTH,160000.000000
3,2006.0,ANGLESEA,770000.000000
4,2006.0,ASCOT VALE,500000.000000
...,...,...,...
20061,2024.0,ԌRΕΕNVΑLΕ,753100.000000
20062,2024.0,ԌΑRFIΕLD,772500.000000
20063,2024.0,ԌΕΕLОNԌ WEST,815000.000000
20064,2024.0,ԌΕΕLОNԌ WΕST,712500.000000


In [21]:
# Remove the leading and trailing whitespaces from the suburb column
final_expanded_df_avg['suburb'] = final_expanded_df_avg['suburb'].str.strip()

# convert 'year' to int
final_expanded_df_avg['year'] = final_expanded_df_avg['year'].astype(int)

In [22]:
create_dir("../data/raw/oldlistings_buy/")
final_expanded_df_avg.to_csv("../data/raw/oldlistings_buy/oldlistings_buy_3_avg.csv", index=False)

Directory already exists: ../data/raw/oldlistings_buy/

